---
 
Author : Halisson S. Gomides - halisson.gomides@gmail.com

Date : 22/01/2021

---

### Conjunto de dados House Sales in King County, USA
>https://www.kaggle.com/harlfoxem/housesalesprediction


### Novas perguntas do CEO:
>1. Qual a data do imóvel mais antigo no portfolio?
>2. Quantos imóveis possuem o número máximo de andares?
>3. Criar uma classificação para os imóveis, separando-os em baixo e alto padrão, de acordo com o preço.
>> Acima de 540.000 -> alto padrão
>> Abaixo de 540.000 -> baixo padrão
>4. Relatório ordenado pelo preço e contendo as seguintes informações:
>> id do imóvel, data que o imóvel ficou disponível para compra, número de quartos, tamanho tota do terreno, preço, a classificação do imóvel (alto e baixo padrão)
>5. Mapa indicando onde as casas estão localizadas geograficamente

In [1]:
import pandas as pd

In [2]:
dfhouses = pd.read_csv('dados/originais/kc_house_data.csv.zip', parse_dates=['date'], infer_datetime_format=True)

In [3]:
dfhouses.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
dfhouses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  int64         
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  int64         
 6   sqft_lot       21613 non-null  int64         
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  int64         
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  int64         
 13  sqft_basement  21613 non-null  int64         
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   2161

In [5]:
# Verificando se existem indices duplicados

for k,v in dfhouses['id'].value_counts().to_dict().items():
    if v > 1: 
        print(k, v)
        print(dfhouses.loc[dfhouses['id'] == k])

        

795000620 3
              id       date     price  bedrooms  bathrooms  sqft_living  \
17602  795000620 2014-09-24  115000.0         3        1.0         1080   
17603  795000620 2014-12-15  124000.0         3        1.0         1080   
17604  795000620 2015-03-11  157000.0         3        1.0         1080   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
17602      6250     1.0           0     0  ...      5        1080   
17603      6250     1.0           0     0  ...      5        1080   
17604      6250     1.0           0     0  ...      5        1080   

       sqft_basement  yr_built  yr_renovated  zipcode      lat    long  \
17602              0      1950             0    98168  47.5045 -122.33   
17603              0      1950             0    98168  47.5045 -122.33   
17604              0      1950             0    98168  47.5045 -122.33   

       sqft_living15  sqft_lot15  
17602           1070        6250  
17603           1070        6250  
17604   

               id       date     price  bedrooms  bathrooms  sqft_living  \
13023  9211500620 2014-10-08  182700.0         3       2.25         1740   
13024  9211500620 2015-04-28  305000.0         3       2.25         1740   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
13023      6650     1.0           0     0  ...      7        1240   
13024      6650     1.0           0     0  ...      7        1240   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
13023            500      1978             0    98023  47.2979 -122.379   
13024            500      1978             0    98023  47.2979 -122.379   

       sqft_living15  sqft_lot15  
13023           1740        7000  
13024           1740        7000  

[2 rows x 21 columns]
5127001320 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
8630  5127001320 2014-11-25  190000.0         3       1.75         1520   
8631  5127001320 2015-02-23  314950.0         

[2 rows x 21 columns]
8832900780 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
6434  8832900780 2014-10-13  480000.0         5        2.0         1760   
6435  8832900780 2015-04-08  647500.0         5        2.0         1760   

      sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
6434     21562     1.0           0     1  ...      8        1560   
6435     21562     1.0           0     1  ...      8        1560   

      sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
6434            200      1959             0    98028  47.7597 -122.263   
6435            200      1959             0    98028  47.7597 -122.263   

      sqft_living15  sqft_lot15  
6434           2150       12676  
6435           2150       12676  

[2 rows x 21 columns]
8564860270 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
20669  8564860270 2014-07-08  449990.0         4        2.5         2680   
20670  8564860270 201

[2 rows x 21 columns]
705730280 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
15299  705730280 2014-08-19  325000.0         3        2.5         1740   
15300  705730280 2015-04-21  335000.0         3        2.5         1740   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
15299      5267     2.0           0     0  ...      7        1740   
15300      5267     2.0           0     0  ...      7        1740   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
15299              0      1999             0    98038  47.3777 -122.023   
15300              0      1999             0    98038  47.3777 -122.023   

       sqft_living15  sqft_lot15  
15299           2180        5000  
15300           2180        5000  

[2 rows x 21 columns]
3333002450 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
14366  3333002450 2014-07-08  165000.0         1        1.0          850   
14367  333300

[2 rows x 21 columns]
2621600015 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
7847  2621600015 2015-01-21  120000.0         3        1.0         1150   
7848  2621600015 2015-04-30  175000.0         3        1.0         1150   

      sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
7847      8924     1.0           0     0  ...      6        1150   
7848      8924     1.0           0     0  ...      6        1150   

      sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
7847              0      1943             0    98030  47.3865 -122.217   
7848              0      1943             0    98030  47.3865 -122.217   

      sqft_living15  sqft_lot15  
7847           1492        8924  
7848           1492        8924  

[2 rows x 21 columns]
4139480200 2
             id       date      price  bedrooms  bathrooms  sqft_living  \
313  4139480200 2014-06-18  1384000.0         4       3.25         4290   
314  4139480200 2014-12

...

              id       date     price  bedrooms  bathrooms  sqft_living  \
4342  9353300600 2014-06-24  348500.0         3        1.5         1360   
4343  9353300600 2015-03-26  370000.0         3        1.5         1360   

      sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
4342     10726     1.0           0     0  ...      7        1360   
4343     10726     1.0           0     0  ...      7        1360   

      sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
4342              0      1966             0    98059  47.4948 -122.134   
4343              0      1966             0    98059  47.4948 -122.134   

      sqft_living15  sqft_lot15  
4342           1650       10726  
4343           1650       10726  

[2 rows x 21 columns]
1524079093 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
4872  1524079093 2014-08-27  275000.0         3       1.75         1300   
4873  1524079093 2015-03-18  369500.0         3       1.75

[2 rows x 21 columns]
6141100320 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
11061  6141100320 2014-07-07  245000.0         2        1.0         1500   
11062  6141100320 2015-02-13  570000.0         2        1.0         1500   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
11061      6685     1.0           0     0  ...      7        1190   
11062      6685     1.0           0     0  ...      7        1190   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
11061            310      1926             0    98133  47.7186 -122.354   
11062            310      1926             0    98133  47.7186 -122.354   

       sqft_living15  sqft_lot15  
11061           1420        6561  
11062           1420        6561  

[2 rows x 21 columns]
6021500970 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
19335  6021500970 2014-05-28  345000.0         2        1.0         1080   
19336  60

[2 rows x 21 columns]
109200390 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
11433  109200390 2014-08-20  245000.0         3       1.75         1480   
11434  109200390 2014-10-20  250000.0         3       1.75         1480   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
11433      3900     1.0           0     0  ...      7        1480   
11434      3900     1.0           0     0  ...      7        1480   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
11433              0      1980             0    98023  47.2977 -122.367   
11434              0      1980             0    98023  47.2977 -122.367   

       sqft_living15  sqft_lot15  
11433           1830        6956  
11434           1830        6956  

[2 rows x 21 columns]
1788900230 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
13756  1788900230 2014-07-22   86500.0         3        1.0          840   
13757  178890

               id       date     price  bedrooms  bathrooms  sqft_living  \
20612  2724049222 2014-08-02  163800.0         2        2.5         1000   
20613  2724049222 2014-12-01  220000.0         2        2.5         1000   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
20612      1092     2.0           0     0  ...      7         990   
20613      1092     2.0           0     0  ...      7         990   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
20612             10      2004             0    98118  47.5419 -122.271   
20613             10      2004             0    98118  47.5419 -122.271   

       sqft_living15  sqft_lot15  
20612           1330        1466  
20613           1330        1466  

[2 rows x 21 columns]
7230400400 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
12030  7230400400 2014-09-26  240000.0         3        2.0         1220   
12031  7230400400 2015-03-26  415500.0      

[2 rows x 21 columns]
6143000020 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
17133  6143000020 2014-10-27  175000.0         3       1.75         1910   
17134  6143000020 2015-04-06  299000.0         3       1.75         1910   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
17133     17003     1.5           0     0  ...      8        1910   
17134     17003     1.5           0     0  ...      8        1910   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
17133              0      1963             0    98001  47.3095 -122.283   
17134              0      1963             0    98001  47.3095 -122.283   

       sqft_living15  sqft_lot15  
17133           1820       14806  
17134           1820       14806  

[2 rows x 21 columns]
3935900232 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
12388  3935900232 2014-09-29  207000.0         3        1.0          920   
12389  39

In [43]:
# Função para apresentar o resultado (pergunta / resposta)
from typing import Union

def resultado(iq:int, resposta:Union[int,float,list], txtcompl:str = '') -> None:
    '''
    Função para apresentar o resultado (pergunta / resposta)
    '''
    
    list_questoes = [
        'Qual a data do imóvel mais antigo no portfolio?',
        'Quantos imóveis possuem o número máximo de andares?',
        'Criar uma classificação para os imóveis, separando-os em baixo e alto padrão, de acordo com o preço.',
        'Relatório ordenado pelo preço.',
        'Mapa indicando onde as casas estão localizadas geograficamente.']
    
    if type(resposta) is list:
        resposta = ', '.join(map(str, resposta))
        
    print(f'{iq+1}. {list_questoes[iq]}\n Resp.: {txtcompl} {resposta}')
    
    


In [8]:
# Removendo as casas com IDs duplicados, mantendo apenas o registro com a data mais recente

df_houses = dfhouses.sort_values(by=['id', 'date'])
# df_order.head(10)
df_houses.drop_duplicates(subset=['id'], keep='last', inplace=True)

# testando o resultado com um id que eu sei que é duplicado
df_houses.query('id == 795000620')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
17604,795000620,2015-03-11,157000.0,3,1.0,1080,6250,1.0,0,0,...,5,1080,0,1950,0,98168,47.5045,-122.33,1070,6250


In [38]:
# Qual a data do imóvel mais antigo no portfolio?

resultado(0, df_houses.nsmallest(1, 'date')['date'].tolist()[0].strftime('%d/%m/%Y'))

1. Qual a data do imóvel mais antigo no portfolio?
 Resp.:  02/05/2014


In [37]:
# Quantos imóveis possuem o número máximo de andares?

resultado(1, df_houses.loc[df_houses['floors'] == df_houses['floors'].max()].shape[0])

2. Quantos imóveis possuem o número máximo de andares?
 Resp.:  8


In [42]:
# Criar uma classificação para os imóveis, separando-os em baixo e alto padrão, de acordo com o preço.

df_houses['padrao'] = df_houses['price'].apply(lambda preco: 'baixo_padrao' if preco < 540000 else 'alto_padrao')

In [45]:
# nº de casas de alto padrão
lst_padrao = [df_houses.query('padrao == "alto_padrao"').shape[0], df_houses.query('padrao == "baixo_padrao"').shape[0]]
lst_padrao[0]

# nº de casas de baixo padrão
lst_padrao[1]

resultado(2, lst_padrao, 'nº de casas de alto padrão e baixo padrão respectivamente.')

3. Criar uma classificação para os imóveis, separando-os em baixo e alto padrão, de acordo com o preço.
 Resp.: nº de casas de alto padrão e baixo padrão respectivamente. 7973, 13463


In [53]:
# Relatório ordenado pelo preço.

# id do imóvel, 
# data que o imóvel ficou disponível para compra, 
# número de quartos, 
# tamanho total do terreno, 
# preço, 
# a classificação do imóvel (alto e baixo padrão)

lst_columns = ['id', 'date', 'bedrooms', 'sqft_lot', 'price', 'padrao']
df_houses.sort_values(by='price')[lst_columns].to_csv('dados/processados/ex02_relatorioCEO.csv', index=False)


In [48]:
# Mapa indicando onde as casas estão localizadas geograficamente.

!pip install plotly

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=94769a85c5159fa92a045c77f436b751aa1e740838b458396fabcfcc4fe08f7e
  Stored in directory: c:\users\halisson\appdata\local\pip\cache\wheels\f9\8d\8d\f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [49]:
import plotly.express as px

In [55]:
lst_columns = ['id', 'lat', 'long', 'price']
mapa = px.scatter_mapbox( df_houses[lst_columns], lat='lat', lon='long',
                 hover_name='id',
                 hover_data=['price'],
                 color_discrete_sequence=['fuchsia'],
                 size='price',
                 height=300)
mapa.update_layout(mapbox_style='open-street-map')
mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
mapa.show()

mapa.write_html('dados/processados/ex02_mapa_house_rocket.html')